Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [2]:
using Plots, ApproxFun

In [3]:
function Base.expm(A::Matrix{T},f::Fun) where {T<:Number}
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [4]:
T = π;a=0.15
t = Fun(identity,0..T)
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ Operator(diagm(fill(ldirichlet(d),n)));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

# we need eye(n) for the initial condition and zeros(n,n) for the
# right-hand side of the ODE

Φ = A\eye(2n,n);  

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [5]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [6]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

5.551115123125783e-16 - 0.2683546905319053im
4.657190431911484e-16 + 0.2683546905319056im
-0.03747531973228899 + 0.9999999999999997im
0.03747531973228914 + 0.9999999999999992im

Floquet Multipliers = 

0.6651802570066617 - 0.7466828146465908im
0.6651802570066608 + 0.7466828146465911im
-0.8889340869219562 + 8.983951009601116e-16im
-1.1249427991479366 + 2.6356405538206627e-15im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [7]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [9]:
t = Fun(identity,0..10T)
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(xsol)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 592.126,225.625 592.122,225.646 592.113,225.687 592.099,225.748 592.082,225.831 592.06,225.933 592.033,226.056 592.002,226.199 591.967,226.362 591.927,226.545 
 591.883,226.748 591.834,226.97 591.781,227.212 591.723,227.472 591.661,227.752 591.595,228.05 591.524,228.366 591.449,228.7 591.369,229.052 591.285,229.421 
 591.197,229.807 591.104,230.209 591.007,230.628 590.905,231.061 590.799,231.51 590.689,231.974 590.574,232.451 590.454,232.943 590.331,233.446 590.203,233.963 
 590.07,234.491 589.933,235.03 589.792,235.579 589.646,236.138 589.496,236.706 589.342,237.282 589.183,237.866 589.02,238.456 588.852,239.052 588.68,239.652 
 588.504,240.257 588.323,240.865 588.138,241.475 587.949,242.087 587.755,242.699 587.557,243.31 587.355,243.919 587.148,244.526 586.937,245.129 586.721,245.727 
 586.501,246.32 586.277,246.905 586.049,247.482 585.816,248.05 585.579,248.608 585.337,249.155 585.091,249.689 584.841,250.21 584.587,250.715 584.328,251.205 
 584.065,251.678 583.798,252.133 583.526,252.569 583.25,252.984 582.97,253.379 582.685,253.751 582.397,254.1 582.103,254.424 581.806,254.723 581.505,254.997 
 581.199,255.243 580.889,255.461 580.574,255.651 580.256,255.812 579.933,255.942 579.606,256.043 579.275,256.111 578.939,256.149 578.6,256.154 578.256,256.127 
 577.908,256.067 577.555,255.975 577.199,255.849 576.838,255.69 576.473,255.499 576.104,255.274 575.731,255.017 575.354,254.728 574.972,254.407 574.586,254.054 
 574.197,253.671 573.803,253.257 573.405,252.814 573.002,252.343 572.596,251.844 572.185,251.319 571.771,250.768 571.352,250.193 570.929,249.596 570.503,248.977 
 570.072,248.338 569.637,247.681 569.198,247.007 568.754,246.318 568.307,245.615 567.856,244.901 567.401,244.178 566.941,243.446 566.478,242.709 566.011,241.967 
 565.539,241.224 565.064,240.48 564.585,239.738 564.101,239 563.614,238.268 563.123,237.543 562.628,236.828 562.128,236.124 561.625,235.432 561.118,234.756 
 560.607,234.095 560.092,233.452 559.574,232.827 559.051,232.223 558.524,231.64 557.994,231.079 557.459,230.541 556.921,230.026 556.379,229.535 555.833,229.067 
 555.283,228.624 554.729,228.204 554.172,227.807 553.61,227.433 553.045,227.08 552.476,226.747 551.904,226.433 551.327,226.135 550.747,225.852 550.163,225.582 
 549.575,225.321 548.984,225.068 548.388,224.818 547.789,224.568 547.187,224.314 546.58,224.053 545.97,223.78 545.356,223.491 544.739,223.181 544.118,222.846 
 543.493,222.479 542.864,222.077 542.232,221.634 541.596,221.143 540.957,220.601 540.314,220.001 539.668,219.338 539.017,218.607 538.364,217.802 537.706,216.918 
 537.046,215.95 536.381,214.893 535.713,213.743 535.042,212.495 534.367,211.147 533.689,209.693 533.007,208.131 532.321,206.458 531.632,204.673 530.94,202.774 
 530.244,200.76 529.545,198.629 528.842,196.384 528.136,194.024 527.427,191.551 526.714,188.967 525.998,186.276 525.278,183.481 524.555,180.586 523.829,177.597 
 523.099,174.519 522.366,171.359 521.63,168.124 520.89,164.822 520.148,161.463 519.401,158.054 518.652,154.607 517.899,151.131 517.144,147.639 516.384,144.142 
 515.622,140.652 514.857,137.182 514.088,133.747 513.316,130.359 512.541,127.034 511.763,123.786 510.982,120.63 510.197,117.583 509.409,114.659 508.619,111.875 
 507.825,109.247 507.028,106.792 506.228,104.525 505.425,102.463 504.619,100.623 503.81,99.0195 502.998,97.6687 502.183,96.5855 501.365,95.7845 500.544,95.2794 
 499.72,95.0832 498.893,95.2079 498.064,95.6642 497.231,96.4617 496.395,97.6084 495.557,99.1109 494.715,100.974 493.871,103.2 493.024,105.79 492.174,108.744 
 491.321,112.056 490.465,115.722 489.607,119.734 488.746,124.079 487.882,128.746 487.015,133.718 486.146,138.977 485.273,144.504 484.399,150.274 483.521,156.263 
 482.641,162.444 481.758,168.788 480.872,175